<a href="https://colab.research.google.com/github/NolanRink/CS4540/blob/main/HW15/HW15_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

placeholder